In [1]:
state = 'WV' 
year = 2010

In [2]:
import sys, os
src_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(src_path)

In [3]:
filepath = '../../dat/' + str(year) + '/'
filename = state + '_county.json'
filename2 = state + '_county.shp'

In [4]:
from read import read_graph_from_json
from epsilon_constraint import*
import networkx as nx

G = read_graph_from_json(state, filepath + filename, year=year)
print(f"The state of {state} has {G._k} districts.")
G._ideal_population = sum(G.nodes[i]['TOTPOP'] for i in G.nodes) / G._k

The state of WV has 3 districts.


In [5]:
#import warm starts
sys.path.append(os.path.abspath('../heuristic'))

from WV_plans_2010 import plans
print(f"Loaded {len(plans)} plans from file.")
warm_starts = plans

Loaded 0 plans from file.


In [6]:
obj_type = 'perimeter'
starting_deviation = 0.01 
verbose = True
contiguity = 'lcut'               # {'lcut', 'scf', 'shir'}
warm_start_mode = 'user'          # {'None', 'user', 'refinement'}
warm_starts = warm_starts                # if you have user define warm strats else it is None
time_limit = 7200                 # Time limit in seconds, adjust as needed
multiplier = 0                    # Adjust multiplier as needed
max_B = True                      # If symmetry_breaking is 'orbitope' or you have warm_start, max_B should be True
symmetry_breaking = 'orbitope'    # {None, 'orbitope', 'rsum'}              
sizes = None

In [ ]:
print(f"Running epsilon constraint method for state {state}, objective {obj_type}")
print(f"{'#' * 100}\n")
(plans, obj_bounds, deviations) = epsilon_constraint_method(
            G,                 
            obj_type,          
            contiguity,        
            cutoff=None,       
            verbose=verbose,
            warm_start_mode = warm_start_mode,
            warm_starts=warm_starts,  
            starting_deviation=starting_deviation, 
            time_limit=time_limit, 
            sizes=sizes,      
            max_B=max_B,      
            symmetry_breaking=symmetry_breaking,
            state=state,
            year=year
        )

Running epsilon constraint method for state WV, objective perimeter
####################################################################################################

L = 611489 and U = 623841

****************************************
Trying deviation = 6176.6466666666665
****************************************
No valid warm start used.

****************************************
Running labeling model!
****************************************
L = 611489 and U = 623841
Set parameter Username
Set parameter LicenseID to value 2608266
Academic license - for non-commercial use only - expires 2026-01-09
sizes =  [1, 1, 1]
Solving the max B problem (as MIP) for use in the vertex ordering...
Set parameter LogToConsole to value 0
Set parameter LazyConstraints to value 1
Set parameter MIPGap to value 0
Set parameter FeasibilityTol to value 1e-07
Set parameter TimeLimit to value 7200
Set parameter IntFeasTol to value 1e-07
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (win64 - Windows 10.0 

H  888   579                      34.7141479   28.62703  17.5%  62.2    2s
* 1149   581              16      34.3182631   29.83847  13.1%  60.3    2s
H 1212   581                      33.2147835   29.83847  10.2%  59.6    2s
H 1404   496                      32.4622401   31.25934  3.71%  59.8    2s

Cutting planes:
  Gomory: 6
  Cover: 6
  Clique: 5
  Zero half: 33
  Lazy constraints: 253

Explored 1968 nodes (93178 simplex iterations) in 2.57 seconds (1.24 work units)
Thread count was 20 (of 20 available processors)

Solution count 6: 32.4622 33.2148 34.3183 ... 38.2312

Optimal solution found (tolerance 0.00e+00)
Best objective 3.246224012802e+01, best bound 3.246224012802e+01, gap 0.0000%

User-callback calls 4744, time in user-callback 0.62 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
[[1, 8, 15, 18, 25, 28, 30, 38, 40, 41, 43, 44, 50, 53], [0, 2, 3, 4, 5, 6, 10, 11, 16, 17, 20, 29, 31, 32, 33, 35, 4

 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   25.76237    0  183          -   25.76237      -     -    0s
     0     0   25.98281    0  191          -   25.98281      -     -    0s
     0     0   25.99140    0  191          -   25.99140      -     -    0s
     0     0   26.21276    0  202          -   26.21276      -     -    0s
     0     0   26.22463    0  205          -   26.22463      -     -    0s
     0     0   26.25109    0  202          -   26.25109      -     -    0s
     0     0   26.25679    0  211          -   26.25679      -     -    0s
     0     0   26.34237    0  228          -   26.34237      -     -    0s
     0     0   26.35110    0  214          -   26.35110      -     -    0s
     0     0   26.35504    0  215          -   26.35504      -     -    0s
     0     0   26.35893    0  222          -   26.35893      -     -    0s
     0     0   26.36157    0  222          -   26.36157      -     -    0s
     0     2   26.3776

H 3167   454                      34.7345355   32.96547  5.09%  64.5    3s

Cutting planes:
  Gomory: 8
  Cover: 26
  Clique: 2
  Inf proof: 2
  Zero half: 10
  RLT: 1
  Lazy constraints: 292

Explored 4158 nodes (248312 simplex iterations) in 3.80 seconds (2.46 work units)
Thread count was 20 (of 20 available processors)

Solution count 5: 34.7345 34.7708 34.8078 ... 38.7027

Optimal solution found (tolerance 0.00e+00)
Best objective 3.473453553811e+01, best bound 3.473453553811e+01, gap 0.0000%

User-callback calls 9108, time in user-callback 0.74 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
[[1, 7, 8, 15, 19, 28, 34, 38, 40, 43, 44, 53, 54], [0, 2, 5, 11, 13, 16, 17, 21, 22, 23, 24, 26, 27, 35, 39, 45, 47, 49], [3, 4, 6, 9, 10, 12, 14, 18, 20, 25, 29, 30, 31, 32, 33, 36, 37, 41, 42, 46, 48, 50, 51, 52]]

****************************************
Trying deviation = 2417.1666666667056
*******************

* 5086   726              27      34.8002525   33.19085  4.62%  52.0    4s
* 6364   289              20      34.7774613   34.03501  2.13%  50.9    4s
  6366     0   34.45943   11  163   34.77746   34.03581  2.13%  50.9    5s

Cutting planes:
  Gomory: 5
  Cover: 42
  Clique: 3
  Inf proof: 6
  Zero half: 25
  Relax-and-lift: 1
  Lazy constraints: 257

Explored 6689 nodes (334159 simplex iterations) in 5.07 seconds (3.47 work units)
Thread count was 20 (of 20 available processors)

Solution count 10: 34.7775 34.8003 35.129 ... 39.092

Optimal solution found (tolerance 0.00e+00)
Best objective 3.477746134842e+01, best bound 3.477746134842e+01, gap 0.0000%

User-callback calls 14390, time in user-callback 0.64 sec

****************************************
Optimal solution found! Gurobi status: 2
****************************************
[[1, 8, 14, 15, 18, 20, 25, 28, 29, 38, 40, 43, 44, 51, 53], [0, 2, 3, 4, 5, 6, 10, 11, 16, 17, 30, 31, 32, 33, 35, 41, 45, 48, 49, 50], [7, 9, 12, 13, 19,

H 4431  1914                      41.3128527   30.08439  27.2%  55.2    4s
  5906  2202   34.41399   25  149   41.31285   31.41770  24.0%  52.8    5s
H 5919  2137                      40.7768610   31.41770  23.0%  52.8    5s
H 5920  2030                      40.0970534   31.41770  21.6%  52.8    5s
H 6187  1442                      37.4107749   31.77181  15.1%  52.9    5s
H 6442  1423                      37.1896866   31.77181  14.6%  52.5    5s
* 6650  1395              47      36.7595251   31.88043  13.3%  51.9    5s
H 7358  1452                      36.4874260   31.95400  12.4%  51.2    5s
H 7714  1430                      36.0398552   32.23957  10.5%  51.0    6s
H 7802  1190                      35.5118953   32.35483  8.89%  50.9    6s
H 7886  1112                      35.1644861   32.35483  7.99%  50.9    6s

Cutting planes:
  Lazy constraints: 123

Explored 13038 nodes (627375 simplex iterations) in 8.10 seconds (4.13 work units)
Thread count was 20 (of 20 available processors)



     0     2   26.36696    0  210          -   26.36696      -     -    0s
H 4412  2737                      44.3168584   30.71483  30.7%  51.2    3s
H 4414  2646                      44.0927585   30.71483  30.3%  51.1    3s
H 5675  2747                      42.8362713   31.31630  26.9%  49.6    4s
H 6367  2184                      38.7565487   31.42112  18.9%  49.1    4s
H 6463  1616                      36.9626633   31.66553  14.3%  49.1    4s
  7677  1871   34.28882   25  164   36.96266   32.02641  13.4%  48.6    5s
 26654  5609   35.79444   34  124   36.96266   33.90954  8.26%  41.8   10s
 50902  5879   36.11439   35   57   36.96266   34.93591  5.48%  40.2   15s
 73244     0 infeasible   35        36.96266   35.96775  2.69%  38.9   20s

Cutting planes:
  Lazy constraints: 83

Explored 74887 nodes (2886815 simplex iterations) in 20.38 seconds (11.30 work units)
Thread count was 20 (of 20 available processors)

Solution count 5: 36.9627 38.7565 42.8363 ... 44.3169

Optimal solution f

     0     0   26.37110    0  208          -   26.37110      -     -    0s
     0     0   26.37572    0  209          -   26.37572      -     -    0s
     0     0   26.37572    0  212          -   26.37572      -     -    0s
     0     0   26.43467    0  217          -   26.43467      -     -    0s
     0     0   26.45409    0  212          -   26.45409      -     -    0s
     0     0   26.45468    0  212          -   26.45468      -     -    0s
     0     0   26.47276    0  228          -   26.47276      -     -    0s
     0     0   26.47934    0  221          -   26.47934      -     -    0s
     0     0   26.48003    0  221          -   26.48003      -     -    0s
     0     0   26.54176    0  239          -   26.54176      -     -    0s
     0     0   26.58336    0  230          -   26.58336      -     -    0s
     0     0   26.58731    0  233          -   26.58731      -     -    0s
     0     0   26.59901    0  233          -   26.59901      -     -    0s
     0     2   26.63174  

     0     0   26.33234    0  196          -   26.33234      -     -    0s
     0     0   26.33601    0  198          -   26.33601      -     -    0s
     0     0   26.35870    0  205          -   26.35870      -     -    0s
     0     0   26.36214    0  207          -   26.36214      -     -    0s
     0     0   26.37121    0  199          -   26.37121      -     -    0s
     0     0   26.38050    0  203          -   26.38050      -     -    0s
     0     0   26.38507    0  204          -   26.38507      -     -    0s
     0     0   26.39279    0  216          -   26.39279      -     -    0s
     0     0   26.39661    0  216          -   26.39661      -     -    0s
     0     0   26.42615    0  216          -   26.42615      -     -    0s
     0     0   26.43334    0  218          -   26.43334      -     -    0s
     0     0   26.45331    0  214          -   26.45331      -     -    0s
     0     0   26.46181    0  212          -   26.46181      -     -    0s
     0     0   26.46181  

In [ ]:
result = list(zip(plans, obj_bounds , deviations))

In [ ]:
# Determine the senses for the Pareto frontier
# Minimize the first objective ('deviation').
# Maximize the second objective for compactness types, otherwise minimize it.
senses = ['min', 'max' if obj_type in ['average_Polsby_Popper','bottleneck_Polsby_Popper'] else 'min']
obj_names = ['deviation_persons', obj_type]
infeasible_region = [0, min(round(r[2],1) for r in result)]

In [ ]:
from pareto import ParetoFrontier

print(f"\n{'#' * 100}\nPareto Frontier for state {state},  objective {obj_type}\n{'#' * 100}\n")
pareto = ParetoFrontier(senses, obj_names, state=state, level='county')

if not  plans:
    print("No plan found!")
else:
    for plan, obj_bound, dev in result:
        upper_bound = [dev,  1/obj_bound[0] if  obj_type == 'bottleneck_Polsby_Popper' else obj_bound[0]]
        lower_bound = [dev,  1/obj_bound[1] if  obj_type == 'bottleneck_Polsby_Popper' else obj_bound[1]]
        
        pareto.add_plan(plan, upper_bound, lower_bound)
            
print("Pareto front plans:", pareto.plans)
print("Pareto front upper bounds:", pareto.upper_bounds)
print("Pareto front lower bounds:", pareto.lower_bounds)
    
#extra_points: list of tuples, each containing (deviation, objective_value, label)
if  year == 2010:
    
    #2010 enacted map scores
    enacted_map_deviation = 3197.333333333372
    enacted_map_scores = {'inverse_Polsby_Popper': 7.75, 'cut_edges': 34.00, 
                                      'perimeter': 42.12, 'average_Polsby_Popper': 0.14,'bottleneck_Polsby_Popper': 0.10}
    # Cooper plan 1
    Cooper_plan_1_deviation = 323.66666666662786
    Cooper_plan_1_scores = {'inverse_Polsby_Popper': 7.31, 'cut_edges': 34.00, 
                                      'perimeter': 40.53, 'average_Polsby_Popper': 0.17,'bottleneck_Polsby_Popper': 0.10}
    # Cooper plan 2
    Cooper_plan_2_deviation = 232.66666666662786
    Cooper_plan_2_scores = {'inverse_Polsby_Popper': 8.18, 'cut_edges': 36.00, 
                                      'perimeter': 43.36, 'average_Polsby_Popper': 0.16,'bottleneck_Polsby_Popper': 0.09}
    # Cooper plan 3
    Cooper_plan_3_deviation = 115.66666666662786
    Cooper_plan_3_scores = {'inverse_Polsby_Popper': 7.25, 'cut_edges': 35.00, 
                                      'perimeter': 40.65, 'average_Polsby_Popper': 0.16,'bottleneck_Polsby_Popper': 0.09}
    extra_points = [
                (enacted_map_deviation,  enacted_map_scores[obj_type], 'Enacted Map'),
                (Cooper_plan_1_deviation,  Cooper_plan_1_scores[obj_type] , 'Cooper plan 1'),
                (Cooper_plan_2_deviation,  Cooper_plan_2_scores[obj_type], 'Cooper plan 2'),
                (Cooper_plan_3_deviation, Cooper_plan_3_scores[obj_type], 'Cooper plan 3'),
                        ]
    
    #extra_colors: list of colors corresponding to the points in extra_points
    extra_colors = ['r', 'g', 'c', 'y']  # Red, Green, Cyan, Yellow for each of the extra points
    
    for ep in extra_points:
        print(f"The {ep[2]} has an objective value of {ep[1]} and a deviation of {ep[0]}.")
else:
    extra_points = None
    extra_colors = None
        
pareto.tighten_lower_bounds()
    
if infeasible_region[1] > 0.5:
    max_dev = max(upper_bound[0] for upper_bound in pareto.upper_bounds)
    max_obj = max(upper_bound[1] for upper_bound in pareto.upper_bounds)
    min_obj = min(upper_bound[1] for upper_bound in pareto.upper_bounds)
    o1lim=[-1, max_dev+100]
    o2lim=[min_obj*0.9, max_obj*(1.1)]
else:
    o1lim = None
    o2lim = None

pareto.plot_with_custom_x_ranges(method = 'epsilon_constraint_method', splits=None, 
                                 o1lim=o1lim, o2lim=o2lim, infeasible_region = infeasible_region,
                                 extra_points=extra_points, extra_colors=extra_colors)

In [ ]:
from draw import *

print(f"\n{'#' * 100}\nPareto maps for state {state}, objective {obj_type}\n{'#' * 100}\n")

format_obj = {
    'bottleneck_Polsby_Popper': lambda x: round(1 /x, 4),
    'cut_edges': lambda x: round(x, 0)}

for plan, obj_bound, dev in result:
    ub = format_obj.get(obj_type, lambda x: round(x, 4))(obj_bound[0])
    lb = format_obj.get(obj_type, lambda x: round(x, 4))(obj_bound[1])
    deviation_percentage = round(100 * dev / G._ideal_population, 4)
    if ub != lb:
        title = f"{round(dev, 2)}-person deviation ({deviation_percentage}%), obj_bounds [{ub}, {lb}] {obj_type}"
    else:
        title = f"{round(dev, 2)}-person deviation ({deviation_percentage}%), with {ub} {obj_type}"

    draw_plan(filepath, filename2, G, plan, title=title, year=year)